In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [69]:
cd "/content/drive/My Drive/DM"

/content/drive/My Drive/DM


In [71]:
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 100)

In [72]:
DATA_PATH = Path.cwd().parent / "data" / "final" / "public"

In [73]:
features_df = pd.read_csv(
   "training_set_features.csv", 
    index_col="respondent_id"
)
labels_df = pd.read_csv(
    "training_set_labels.csv", 
    index_col="respondent_id"
)

In [74]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.preprocessing import OneHotEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.neural_network import MLPClassifier
RANDOM_SEED = 8    # Set a random seed for reproducibility!


from sklearn.preprocessing import OneHotEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.preprocessing import FunctionTransformer

from numpy import nan
import copy

In [75]:
numeric_cols = features_df.columns[features_df.dtypes != "object"].values
non_numeric_cols = features_df.columns[features_df.dtypes == 'object'].values

In [76]:
# create age groups numeric values

seperate_processing_cols = ["age_group", "income_poverty"]
#seperate_processing_cols = []

age_mean_map = {'18 - 34 Years': 26,
                '35 - 44 Years': 40,
                '45 - 54 Years': 50,
                '55 - 64 Years': 60,
                '65+ Years': 70, 
                }

income_map = {'Below Poverty': 1,
              '<= $75,000, Above Poverty': 2,
              '> $75,000':3
             }

full_map = {"age_group":age_mean_map,
            "income_poverty":income_map}

def map_to_numeric(x, mapping):
    if mapping.get(x) == None:
        return nan
    else:
        return mapping.get(x)

def convert(df):
    new_df = copy.deepcopy(df)
    for col in seperate_processing_cols:
        new_df[col] = df[col].apply(lambda x: map_to_numeric(x, full_map.get(col))) 
    return new_df

numeric_to_cat_transofrmer = FunctionTransformer(convert)   
convert(features_df)["income_poverty"].unique()
#features_df


array([ 1.,  2.,  3., nan])

In [77]:
# chain preprocessing into a Pipeline object
# each step is a tuple of (name you chose, sklearn transformer)
high_missing_cols = []
non_numeric_cols = [x for x in non_numeric_cols if (x not in high_missing_cols and x not in seperate_processing_cols)]
numeric_cols = [x for x in numeric_cols if (x not in high_missing_cols and x not in seperate_processing_cols)]
# chain preprocessing into a Pipeline object
# each step is a tuple of (name you chose, sklearn transformer)
numeric_to_cat_preprocessing_steps = Pipeline([
    ('convert_to_cat', numeric_to_cat_transofrmer),
    ('simple_imputer', SimpleImputer(strategy='mean')),
    ('standard_scaler', StandardScaler()),
    ])

numeric_preprocessing_steps = Pipeline(steps=[
    ('standard_scaler', StandardScaler()),
    ('imputer', SimpleImputer(strategy='mean'))])
non_numeric_preprocessing_steps = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
# create the preprocessor stage of final pipeline
# each entry in the transformer list is a tuple of
# (name you choose, sklearn transformer, list of columns)
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        ("non_numeric",non_numeric_preprocessing_steps,non_numeric_cols),
        ("numeric_to_cat", numeric_to_cat_preprocessing_steps, seperate_processing_cols)
    ],
    remainder = "drop"
)

In [78]:
est_mlp = MLPClassifier(hidden_layer_sizes=(200,100,), learning_rate='invscaling', learning_rate_init=0.01,
                          power_t=0.5, max_iter=500, shuffle=True, tol=0.0001, 
                          early_stopping=True, validation_fraction=0.1, n_iter_no_change=10, max_fun=15000,
                         random_state=RANDOM_SEED)
est_xgb = XGBClassifier(learning_rate =0.05, n_estimators=350, max_depth=5, min_child_weight=2, gamma=0.2,
                             subsample=0.8,colsample_bytree=0.4,reg_alpha=0.1,objective= 'binary:logistic',nthread=4,
                              scale_pos_weight=1 ,reg_lambda = 10 , random_state = 42)
est_gb = GradientBoostingClassifier(random_state=8, n_estimators=250)
est_rn = RandomForestClassifier(max_depth=10, bootstrap=True, n_estimators=200)
est_svc = SVC()
est_nb = GaussianNB()
est_kn = KNeighborsClassifier()

estimators_stacked = [('xbg', est_xgb),
              ('mlp', est_mlp),
              #('gb', est_gb),
              #('rn', est_rn)
             ]

stacked_estimator =  StackingClassifier(estimators=estimators_stacked,
                                    final_estimator=LogisticRegression(),
                                    stack_method='predict_proba'
                                  )

In [79]:
estimators = MultiOutputClassifier(
    estimator = stacked_estimator
)

In [80]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

In [81]:
X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=RANDOM_SEED
)

In [82]:
%%time

# Train model
full_pipeline.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict_proba(X_eval)
preds

CPU times: user 4min 32s, sys: 39.8 s, total: 5min 12s
Wall time: 2min 46s


In [83]:
print("test_probas[0].shape", preds[0].shape)
print("test_probas[1].shape", preds[1].shape)

test_probas[0].shape (8814, 2)
test_probas[1].shape (8814, 2)


In [84]:
y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": preds[0][:, 1],
        "seasonal_vaccine": preds[1][:, 1],
    },
    index = y_eval.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (8814, 2)


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
3610,0.061461,0.124360
12424,0.067567,0.092741
11459,0.057021,0.899610
11896,0.053468,0.077307
26524,0.216546,0.586689


In [85]:
roc_auc_score(y_eval, y_preds)

0.8723445115757842

In [86]:
%%time 

full_pipeline.fit(features_df, labels_df)

None   # So we don't print out the whole pipeline representation

CPU times: user 6min 45s, sys: 57.9 s, total: 7min 43s
Wall time: 4min 5s


In [87]:
test_features_df = pd.read_csv(
    "test_set_features.csv", 
    index_col="respondent_id"
)

In [88]:
test_probas = full_pipeline.predict_proba(test_features_df)

In [90]:
submission_df = pd.read_csv(
    "submission_format.csv", 
    index_col="respondent_id"
)

In [91]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = test_probas[0][:, 1]
submission_df["seasonal_vaccine"] = test_probas[1][:, 1]

submission_df.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.090501,0.232599
26708,0.057752,0.074792
26709,0.150042,0.742123
26710,0.765277,0.879990
26711,0.236953,0.473138


In [92]:
submission_df.to_csv('my_submission18.csv', index=True)

In [93]:
!head my_submission18.csv

respondent_id,h1n1_vaccine,seasonal_vaccine
26707,0.09050116886194472,0.23259908434002158
26708,0.057752015650933235,0.07479249211388111
26709,0.15004209108647118,0.7421232872519187
26710,0.7652773409418057,0.8799902116167749
26711,0.23695286849422767,0.47313788894521086
26712,0.895438545447382,0.9276646895560107
26713,0.1695984580981741,0.6749875268285874
26714,0.1263184827056361,0.12358856894310274
26715,0.05520718863258898,0.10436170509975856
